### DML : 데이터 조작언어
- 데이터를 조작 (선택, 삽입, 수정, 삭제)
- DML 구문이 사용되는 대상은 테이블의 행
- DML이 사용되기 위해서는 꼭 이전에 테이블이 정의
- SQL문중 SELECT, INSERT, UPDATE, DELETE가 이 구문에 해당
- 트랜잭션이 발생하는 SQL도 DML에 속함
- 테이블의 데이터를 변경할 때 실제 테이블에 완전히 적용하지 않고, 임시로 적용하는 것
- 취소가능

### DDL : 데이터 정의 언어
- 데이터베이스, 테이블 뷰, 인덱스 등의 데이터 베이스 개체를 생성/삭제/변경
- CREATE, DROP, ALTER구문
- DDL은 트랜잭션 발생 없음
- ROLLBACK이나 COMMIT 사용 불가
- DDL문은 실행 즉시 MySQL에 적용

### DCL : 데이터 제어 언어
- 사용자에게 어떤 권한을 부여하거나 빼앗을 때 주로 사용


In [2]:
library(tidyverse)
library(RMySQL)

In [9]:
conn <- dbConnect(
  MySQL(),
  user = 'root', 
  password = pass, 
  host = 'localhost',
  client.flag = CLIENT_MULTI_RESULTS
)

### ```SHOW DATABASES``` : 현재 서버의 DB확인
### ```USE {DB}``` : 사용할 DB 지정
- 지정해 놓은 후 특별히 다시 USE문 사용하거나 다른 DB를 사용한다는 명시가 없는 이상 모든 SQL문은 지정 DB에서 수행

In [12]:
dbGetQuery(conn, "SHOW DATABASES") %>% print
dbGetQuery(conn, "USE world")

             Database
1  information_schema
2           market_db
3               mysql
4            naver_db
5          netflix_db
6  performance_schema
7              sakila
8             shop_db
9              solodb
10                sys
11              world


<0 x 0 matrix>

### ```SHOW TABLES``` : DB의 테이블 이름 보기
### ```SHOW TABLE STATUS``` : DB의 테이블 정보 조회
### ```DESCRIBE {테이블}``` : 테이블에 구조 확인 (DESC)

In [21]:
dbGetQuery(conn, "SHOW TABLES") 
dbGetQuery(conn, "SHOW TABLE STATUS") 
dbGetQuery(conn, "DESCRIBE city") 

Tables_in_world
<chr>
city
country
countrylanguage


Warning message in .local(conn, statement, ...):
"unrecognized MySQL field type 7 in column 11 imported as character"


Name,Engine,Version,Row_format,Rows,Avg_row_length,Data_length,Max_data_length,Index_length,Data_free,Auto_increment,Create_time,Update_time,Check_time,Collation,Checksum,Create_options,Comment
<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
city,InnoDB,10,Dynamic,4188,97,409600,0,131072,0,4079,2022-01-17 20:46:41,NA,NA,latin1_swedish_ci,NA,,
country,InnoDB,10,Dynamic,239,411,98304,0,0,0,NA,2022-01-17 20:46:44,NA,NA,latin1_swedish_ci,NA,,
countrylanguage,InnoDB,10,Dynamic,984,99,98304,0,65536,0,NA,2022-01-17 20:46:44,NA,NA,latin1_swedish_ci,NA,,


Field,Type,Null,Key,Default,Extra
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ID,int,NO,PRI,NA,auto_increment
Name,char(35),NO,,,
CountryCode,char(3),NO,MUL,,
District,char(20),NO,,,
Population,int,NO,,0,


## SELECT 
### ```SELECT ~ FROM```

In [25]:
dbGetQuery(conn, "SELECT * FROM city") # city 테이블 전체 확인
dbGetQuery(conn, "SELECT Name FROM city") # city 테이블 Name열만 확인
dbGetQuery(conn, "SELECT Name, Population FROM city") # 여러개의 열 확인

,ID,Name,CountryCode,District,Population
,<int>,<chr>,<chr>,<chr>,<int>
1,1,Kabul,AFG,Kabol,1780000


,Name
,<chr>
1,Kabul


,Name,Population
,<chr>,<int>
1,Kabul,1780000


### ```WHERE``` 조건절
- 조회하는 결과에 특정한 조건으로 원하는 데이터만 확인 

In [28]:
dbGetQuery(conn,
          "SELECT * FROM city
           WHERE Population >= 800000") %>% head(3)

,ID,Name,CountryCode,District,Population
,<int>,<chr>,<chr>,<chr>,<int>
1,1,Kabul,AFG,Kabol,1780000
2,35,Alger,DZA,Alger,2168000
3,56,Luanda,AGO,Luanda,2022000


### ```AND```, ```OR``` : 관계 연산자, 논리 연산자
- 관계 연산자 (OR, AND, =, <, >, <=, >=, <>, !=)

In [31]:
dbGetQuery(conn, "SELECT * FROM city WHERE CountryCode = 'KOR';") 
# 한국 도시들 보기

dbGetQuery(conn, 
           "SELECT * FROM city 
            WHERE COuntryCode = 'USA'
            AND Population >= 1000000;")
# 인구 100만 이상의 미국 도시들 보기

,ID,Name,CountryCode,District,Population
,<int>,<chr>,<chr>,<chr>,<int>
1,2331,Seoul,KOR,Seoul,9981619
2,2332,Pusan,KOR,Pusan,3804522
3,2333,Inchon,KOR,Inchon,2559424


,ID,Name,CountryCode,District,Population
,<int>,<chr>,<chr>,<chr>,<int>
1,3793,New York,USA,New York,8008278
2,3794,Los Angeles,USA,California,3694820
3,3795,Chicago,USA,Illinois,2896016


### ```BETWEEN ~ AND``` : 수치형 데이터 구간

In [33]:
dbGetQuery(conn, "SELECT * FROM city
                  WHERE Population BETWEEN 7000000 AND 8000000;") 

ID,Name,CountryCode,District,Population
<int>,<chr>,<chr>,<chr>,<int>
456,London,GBR,England,7285000
1025,Delhi,IND,Delhi,7206704
1532,Tokyo,JPN,Tokyo-to,7980230
1891,Peking,CHN,Peking,7472000


### ```IN``` : 이산형 데이터 조건 표시

In [41]:
# 한국, 미국, 일본의 도시 
dbGetQuery(conn, "SELECT * FROM city
                  WHERE CountryCode IN('KOR', 'JAP', 'USA')") 

,ID,Name,CountryCode,District,Population
,<int>,<chr>,<chr>,<chr>,<int>
1,2331,Seoul,KOR,Seoul,9981619
2,2332,Pusan,KOR,Pusan,3804522
3,2333,Inchon,KOR,Inchon,2559424


### ```LIKE``` : 문자열의 내용 검색
- 문자뒤에 '%' : 해당 문자로 시작하는 데이터 조회
- 문자앞에 '_' : 해당 문자로 끝나는 데이터 조회

In [43]:
# Name이 Tel로 시작하는 데이터
dbGetQuery(conn, "SELECT * FROM city
                  WHERE Name LIKE 'Tel %'")

ID,Name,CountryCode,District,Population
<int>,<chr>,<chr>,<chr>,<int>
1451,Tel Aviv-Jaffa,ISR,Tel Aviv,348100


In [45]:
# CountryCode가 KO로 시작하는 데이터 
dbGetQuery(conn, "SELECT * FROM city
                  WHERE CountryCode LIKE 'KO_'") 

,ID,Name,CountryCode,District,Population
,<int>,<chr>,<chr>,<chr>,<int>
1,2331,Seoul,KOR,Seoul,9981619
2,2332,Pusan,KOR,Pusan,3804522
3,2333,Inchon,KOR,Inchon,2559424


### ```ORDER BY``` : 결과가 출력되는 순서를 조정 (기본적으로 오름차순)
- 오름차순 : ASC / 내림차순 : DESC

In [56]:
# 인구수로 내림차순하여 한국의 도시 인구수 보기
dbGetQuery(conn, "SELECT * FROM city
                  WHERE CountryCode = 'Kor'
                  ORDER BY Population DESC") 

,ID,Name,CountryCode,District,Population
,<int>,<chr>,<chr>,<chr>,<int>
1,2331,Seoul,KOR,Seoul,9981619
2,2332,Pusan,KOR,Pusan,3804522
3,2333,Inchon,KOR,Inchon,2559424


### ```DISTINCT``` : 중복된 것은 1개씩만 보이며 출력 (테이블의 크기가 클수록 효율적)

In [67]:
dbGetQuery(conn,"SELECT DISTINCT CountryCode FROM city") %>% head

,CountryCode
,<chr>
1,ABW
2,AFG
3,AGO
4,AIA
5,ALB
6,AND


### LIMIT : 출력 개수를 제한 (상위 N개만 출력)
- 서버의 처리량을 많이 사용해 서버의 전반적인 성능을 나쁘게 하는 악성 쿼리문 개선할 때 사용 
- 상단 Limit to **Rows를 통해 수정

In [69]:
dbGetQuery(conn, "SELECT * FROM city
                  WHERE CountryCode = 'Kor'
                  ORDER BY Population DESC 
                  LIMIT 1") 

ID,Name,CountryCode,District,Population
<int>,<chr>,<chr>,<chr>,<int>
2331,Seoul,KOR,Seoul,9981619


### ```GROUP BY``` : 데이터 그룹화
- 집계함수와 함께 사용 

#### 집계 함수 : WHERE절과 함께 사용 불가
- (AVG, MIN, MAX, COUNT, COUNT(DISTINCT), STDEV, VARIANCE)

In [77]:
# 도시들의 평균 인구수 (평균 열 이름은 'MEAN'으로)
dbGetQuery(conn, "SELECT Name, AVG(Population) AS 'MEAN' FROM city
                  GROUP BY Name") 

,Name,MEAN
,<chr>,<dbl>
1,Kabul,1780000
2,Qandahar,237500
3,Herat,186800


In [80]:
# 데이터의 도시 개수
dbGetQuery(conn, "SELECT COUNT(*) AS 'City Count' FROM city")

City Count
<dbl>
4079


### ```HAVING``` : 집계함수에 대한 조건절
- HAVING절은 반드시 GROUP BY절 다음 

In [83]:
# Population의 최댓값이 1000만 이상인 국가들만 표시
dbGetQuery(conn, "SELECT CountryCode, MAX(Population) FROM city
                  GROUP BY CountryCode
                  HAVING MAX(Population) > 10000000")

CountryCode,MAX(Population)
<chr>,<int>
IND,10500000


### ```ROLL UP``` : 총합 도는 중간합계
- GROUP BY절과 함께 WITH ROLLUP문 사용

In [86]:
# 국가별 도시에 대한 합계
dbGetQuery(conn, "SELECT CountryCode, Name, SUM(Population) FROM city
 GROUP BY CountryCode, Name WITH ROLLUP")

,CountryCode,Name,SUM(Population)
,<chr>,<chr>,<dbl>
1,ABW,Oranjestad,29034
2,ABW,NA,29034
3,AFG,Herat,186800
4,AFG,Kabul,1780000
5,AFG,Mazar-e-Sharif,127800
6,AFG,Qandahar,237500
7,AFG,NA,2332100


## Sub Query : 서브쿼리, 쿼리문 안에 또 쿼리문이 포함
- 서브쿼리의 결과가 둘 이상이 되면 에러

In [47]:
# Name이 Seoul인 데이터의 CountryCode = KOR => CountryCode가 KOR인 모든 데이터 확인 
dbGetQuery(conn, "SELECT * FROM city
 WHERE CountryCode = (SELECT CountryCode FROM city 
                      WHERE Name = 'Seoul')")

,ID,Name,CountryCode,District,Population
,<int>,<chr>,<chr>,<chr>,<int>
1,2331,Seoul,KOR,Seoul,9981619
2,2332,Pusan,KOR,Pusan,3804522
3,2333,Inchon,KOR,Inchon,2559424


### ```ANY``` : 서브쿼리의 여러 결과 중 한 가지만 만족해도 가능
(SOME은 ANY와 동일한 의미)

In [50]:
dbGetQuery(conn, "SELECT * FROM city
                  WHERE Population > ANY(SELECT Population FROM city
                                         WHERE District = 'New York')") 

,ID,Name,CountryCode,District,Population
,<int>,<chr>,<chr>,<chr>,<int>
1,206,S횄짙o Paulo,BRA,S횄짙o Paulo,9968485
2,939,Jakarta,IDN,Jakarta Raya,9604900
3,1024,Mumbai (Bombay),IND,Maharashtra,10500000


### ```ALL``` : 서브쿼리의 여러 결과를 모두 만족 시켜야 함

In [51]:
# District가 New York인 모든 데이터의 Population보다 큰 Population값을 가진 데이터 확인 
dbGetQuery(conn, "SELECT * FROM city
                  WHERE Population > ALL(SELECT Population FROM city
                                         WHERE District = 'New York')") 

,ID,Name,CountryCode,District,Population
,<int>,<chr>,<chr>,<chr>,<int>
1,206,S횄짙o Paulo,BRA,S횄짙o Paulo,9968485
2,939,Jakarta,IDN,Jakarta Raya,9604900
3,1024,Mumbai (Bombay),IND,Maharashtra,10500000


## MySQL 내장함수
### 1. 문자열 함수

### ```LENGTH``` : 문자열 길이
### ```CONCAT``` : 문자열 결합
### ```LOCATE``` : 문자열 위치 반환
### ```REPLACE``` : 특정 문자열을 대체 문자열로 교체

In [90]:
dbGetQuery(conn, "SELECT LENGTH('abcdefg'), 
                  CONCAT('My', 'sql Op', 'en Source'), 
                  LOCATE('ABC', '123ABC12341234'),
                  REPLACE('MSSQL', 'MS', 'My');")

LENGTH('abcdefg'),"CONCAT('My', 'sql Op', 'en Source')","LOCATE('ABC', '123ABC12341234')","REPLACE('MSSQL', 'MS', 'My')"
<dbl>,<chr>,<dbl>,<chr>
7,Mysql Open Source,4,MySQL


### ```LEFT``` / ```RIGHT``` : 좌/우 기준 위치의 문자 반환

In [92]:
# 문자열의 좌/우측부터 지정한 개수만큼 문자 반환
dbGetQuery(conn, "SELECT LEFT('MySQL is an open source', 5),
                         RIGHT('MySQL is an open source', 5);")

"LEFT('MySQL is an open source', 5)","RIGHT('MySQL is an open source', 5)"
<chr>,<chr>
MySQL,ource


### ```LOWER``` / ```UPPER``` : 대문자/소문자 변환

In [97]:
dbGetQuery(conn, "SELECT LOWER('MySQL is an open source'),
                         UPPER('MySQL is an open source'); ")

LOWER('MySQL is an open source'),UPPER('MySQL is an open source')
<chr>,<chr>
mysql is an open source,MYSQL IS AN OPEN SOURCE


### ```FLOOR``` / ```CEIL``` / ```ROUND``` : 올림/내림/반올림

In [98]:
dbGetQuery(conn, "SELECT FLOOR(10.95), CEIL(10.95), ROUND(10.95);")

FLOOR(10.95),CEIL(10.95),ROUND(10.95)
<dbl>,<dbl>,<dbl>
10,11,11


### ```TRIM``` :  문자열의 앞이나 뒤, 또는 양쪽 모두에 있는 특정 문자를 제거 
1. BOTH : 전달받은 문자열의 양 끝에 존재하는 특정 문자를 제거
2. LEADING : **전달받은 문자열 앞에 존재하는 특정 문자를 제거**
3. TRAILING : **전달받은 문자열 뒤에 존재하는 특정 문자를 제거**
- 제거할 문자를 명시하지 않으면, 자동으로 공백을 제거  

In [112]:
dbGetQuery(conn, "SELECT TRIM('  MySQL  '),
                         TRIM(LEADING '-' FROM '--MySQL--'),
                         TRIM(TRAILING '-' FROM '--MySQL--')")

TRIM(' MySQL '),TRIM(LEADING '-' FROM '--MySQL--'),TRIM(TRAILING '-' FROM '--MySQL--')
<chr>,<chr>,<chr>
MySQL,MySQL--,--MySQL


### ```FORMAT``` : 숫자 타입의 데이터를 세 자리마다 쉼표를 사용하는 형식으로 변환 

In [114]:
dbGetQuery(conn, "SELECT FORMAT('123123123123123123123', 7);") 
# 7자리 소수점까지 표시 

"FORMAT('123123123123123123123', 7)"
<chr>
"123,123,123,123,123,130,000.0000000"


### 2. 수학 함수

### ```SORT``` : 양의 제곱근
### ```POW``` : 거듭 제곱
### ```EXP``` : 지수의 거듭 제곱
### ```LOG``` : 자연로그 값

In [115]:
dbGetQuery(conn, "SELECT SQRT(3), POW(2, 3), EXP(3), LOG(3);")

SQRT(3),"POW(2, 3)",EXP(3),LOG(3)
<dbl>,<dbl>,<dbl>,<dbl>
1.732051,8,20.08554,1.098612


### ```SIN``` / ```COS``` / ```TAN``` : 삼각함수 

In [119]:
dbGetQuery(conn, "SELECT SIN(PI() / 2), COS(PI() / 2), TAN(PI() / 2);")

SIN(PI() / 2),COS(PI() / 2),TAN(PI() / 2)
<dbl>,<dbl>,<dbl>
1,6.123234e-17,1.633124e+16


### ```ABS```, ```RAND``` : 절댓값 / 0 ~ 1사이 랜덤 실수

In [121]:
dbGetQuery(conn, "SELECT ABS(-5), RAND(), ROUND(RAND()*100, 3);")

ABS(-5),RAND(),"ROUND(RAND()*100, 3)"
<dbl>,<dbl>,<dbl>
5,0.7833786,97.472


### 3. 날짜와 시간 함수

### ```NOW``` : 현재 날짜와 시간을 반환 (YYYY-MM-DD HH:MM:SS | YYYYMMDDHHMMSS)
### ```CURDATE``` : 현재 날짜를 반환 (YYYY-MM-DD | YYYYMMDD)
### ```CURTIME``` : 현재 시각을 반환 (HH:MM:SS | HHMMSS)
### ```CURRENT_TIMESTAMP``` : 오늘 날짜 및 시간
### ```DATEDIFF``` : 시간 차이

In [125]:
dbGetQuery(conn, "SELECT NOW(), CURDATE(), CURTIME(), 
                         DATEDIFF(CURDATE(), '2021-06-25');")

NOW(),CURDATE(),CURTIME(),"DATEDIFF(CURDATE(), '2021-06-25')"
<chr>,<chr>,<chr>,<dbl>
2022-06-25 00:15:14,2022-06-25,00:15:14,365


### ```DATE```/ ```MONTH```/ ```DAY```/ ```HOUR```/ ```MINUTE```/ ```SECOND```

In [123]:
dbGetQuery(conn, "SELECT DATE(NOW()), MONTH(NOW()), DAY(NOW()), HOUR(NOW()), MINUTE(NOW()), SECOND(NOW());")

DATE(NOW()),MONTH(NOW()),DAY(NOW()),HOUR(NOW()),MINUTE(NOW()),SECOND(NOW())
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022-06-25,6,25,0,14,22


### ```MONTHNAME``` / ```DAYNAME```

In [124]:
dbGetQuery(conn, "SELECT MONTHNAME(CURDATE()), DAYNAME(CURDATE());")

MONTHNAME(CURDATE()),DAYNAME(CURDATE())
<chr>,<chr>
June,Saturday


### ```DAYOFWEEK``` : 일자가 해당 주에서 변번째 날인지 변환
### ```DAYOFMONTH``` : 일자가 해당 월에서 몇번째 날인지 변환
### ```DAYOFYEAR``` : 일자가 해당 연도에서 몇번째 날인지 반환

In [126]:
dbGetQuery(conn, "SELECT DAYOFWEEK(CURDATE()), DAYOFMONTH(CURDATE()), DAYOFYEAR(CURDATE());")

DAYOFWEEK(CURDATE()),DAYOFMONTH(CURDATE()),DAYOFYEAR(CURDATE())
<dbl>,<dbl>,<dbl>
7,25,176


### ```DATE_FORMAT``` : 전달받은 형식에 맞춰 날짜와 시간정보를 문자열ㄹ로 반환

In [130]:
dbGetQuery(conn, "SELECT DATE_FORMAT(NOW(), '%D %M %Y');")

"DATE_FORMAT(NOW(), '%D %M %Y')"
<chr>
25th June 2022
